# Part 7

Univeral code that we will use for the entire notebook.

In [1]:
from openai import OpenAI

# Create an instance of the OpenAI class
# This assumes you have the OPENAI_API_KEY environment variable set
client = OpenAI()

## Creating Assistants, Threads, and Messages Review
Let's create a new assistant, thread, and some messages for us to use later on and to review the code for creating them.

### Creating an Assistant
First, let's make an Assistant we can use to communicate with our run.

In [2]:

# Create an assistant.
assistant = client.beta.assistants.create(
    model="gpt-4-turbo",
    instructions="You are a helpful assistant.",
    name="Run Tester Assistant",
    metadata={
        "holds_threads": "True",
        "likes_threads": "True",
        "holds_messages": "True",
        "likes_messages": "True",
    },
    temperature=1,
    top_p=1,
)

# Print the details of the created assistant to check the properties.
print(assistant)
print("\n\n")
print(assistant.name)
print(assistant.metadata)

Assistant(id='asst_kTzVAfsliuawpoR9jpSCXtTd', created_at=1715546726, description=None, instructions='You are a helpful assistant.', metadata={'holds_threads': 'True', 'likes_threads': 'True', 'holds_messages': 'True', 'likes_messages': 'True'}, model='gpt-4-turbo', name='Run Tester Assistant', object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)



Run Tester Assistant
{'holds_threads': 'True', 'likes_threads': 'True', 'holds_messages': 'True', 'likes_messages': 'True'}


### Creating a Thread
Now, let's create a Thread that can be used to hold our messages.

In [3]:
# Create a thread using the OpenAI API and store it in a variable
# The metadata specifies a user identifier
thread = client.beta.threads.create(
    metadata={
        "user": "abc123"
    }
)

# Output the result of the thread creation to the console
print(thread)


Thread(id='thread_5F1SXQGcLp8JMfsSc2O3nfFX', created_at=1715546727, metadata={'user': 'abc123'}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


### Creating a Message
Finally, let's create a Message that we can go into the Thread for use later.

In [4]:
# Create a message in a specific thread using the client's message creation method.
message = client.beta.threads.messages.create(
    thread_id=thread.id,  # ID of the thread where the message will be posted
    role="user",  # Role of the entity posting the message
    content="What is a penguin?",  # The textual content of the message
    metadata={"key": "value"}  # Additional data associated with the message in key-value pairs
)

# Print the entire message object to view its details.
print(message)

# Print a blank line for better readability of the output.
print("\n")

# Print specific attributes of the message.
print(message.id)  # The unique identifier of the message
print(message.content)  # The content of the message
print(message.content[0].text.value)  # Assuming 'content' is a list of text objects, print the value of the first one
print(message.role)  # The role associated with the message

Message(id='msg_d44yF4oH11Fx713GMlpRmRel', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is a penguin? Avoid repeating information.'), type='text')], created_at=1715546727, incomplete_at=None, incomplete_details=None, metadata={'key': 'value'}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_5F1SXQGcLp8JMfsSc2O3nfFX')


msg_d44yF4oH11Fx713GMlpRmRel
[TextContentBlock(text=Text(annotations=[], value='What is a penguin? Avoid repeating information.'), type='text')]
What is a penguin? Avoid repeating information.
user


## Creating Runs
Runs are the engine that makes things happen with the LLM. 

In [5]:

run = client.beta.threads.runs.create(
  assistant_id=assistant.id,
  thread_id=thread.id,
)

print(run)


Run(id='run_QKbVfc577O4ea1eBP36y1rNi', assistant_id='asst_kTzVAfsliuawpoR9jpSCXtTd', cancelled_at=None, completed_at=None, created_at=1715546727, expires_at=1715547327, failed_at=None, incomplete_details=None, instructions='You are a helpful assistant.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-turbo', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_5F1SXQGcLp8JMfsSc2O3nfFX', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})


### Getting the Run Status
You can't just keep looking at the run result to get the current status

In [6]:
print(run)
print("\n")
print(run.status)

Run(id='run_QKbVfc577O4ea1eBP36y1rNi', assistant_id='asst_kTzVAfsliuawpoR9jpSCXtTd', cancelled_at=None, completed_at=None, created_at=1715546727, expires_at=1715547327, failed_at=None, incomplete_details=None, instructions='You are a helpful assistant.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-turbo', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_5F1SXQGcLp8JMfsSc2O3nfFX', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})


queued


You have to get the run status by retrieving the run manually

In [7]:
runstatus = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id,
)

print(runstatus)
print("\n")
print(runstatus.id)
print(runstatus.last_error)
print(runstatus.status)

Run(id='run_QKbVfc577O4ea1eBP36y1rNi', assistant_id='asst_kTzVAfsliuawpoR9jpSCXtTd', cancelled_at=None, completed_at=None, created_at=1715546727, expires_at=1715547327, failed_at=None, incomplete_details=None, instructions='You are a helpful assistant.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-turbo', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_5F1SXQGcLp8JMfsSc2O3nfFX', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})


run_QKbVfc577O4ea1eBP36y1rNi
None
queued


Or you can poll the status manually with a loop

In [9]:
import openai
import time

another_run = client.beta.threads.runs.create(
  assistant_id=assistant.id,
  thread_id=thread.id,
)

# Retrieve initial run status
run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=another_run.id)

# Poll the status while it's still queued or in progress
while run_status.status in ["queued", "in_progress"]:
    # Log current status
    print(f"Run status: {run_status.status}")
    
    # Wait for 1 second
    time.sleep(1)
    
    # Retrieve the status again
    run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=another_run.id)

# Optionally, print the final status or any other detail
print("Final status:", run_status.status)


Run status: queued
Run status: in_progress
Run status: in_progress
Run status: in_progress
Run status: in_progress
Final status: completed


The best option is to save yourself some code and just use the "create_and_poll" method to do the same thing. 

When interacting with the API some actions such as starting a Run and adding files to vector stores are asynchronous and take time to complete. The SDK includes helper functions which will poll the status until it reaches a terminal state and then return the resulting object. If an API method results in an action which could benefit from polling there will be a corresponding version of the method ending in '_and_poll'.

In [10]:
auto_run_and_poll = client.beta.threads.runs.create_and_poll(
    assistant_id=assistant.id,
    thread_id=thread.id,
)

print(auto_run_and_poll)
print("\n")
print(auto_run_and_poll.id)
print(auto_run_and_poll.status)

Run(id='run_lXDSChTpnFObrPkoTf8M2EsC', assistant_id='asst_kTzVAfsliuawpoR9jpSCXtTd', cancelled_at=None, completed_at=1715546755, created_at=1715546751, expires_at=None, failed_at=None, incomplete_details=None, instructions='You are a helpful assistant.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-turbo', object='thread.run', required_action=None, response_format='auto', started_at=1715546751, status='completed', thread_id='thread_5F1SXQGcLp8JMfsSc2O3nfFX', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=Usage(completion_tokens=108, prompt_tokens=246, total_tokens=354), temperature=1.0, top_p=1.0, tool_resources={})


run_lXDSChTpnFObrPkoTf8M2EsC
completed


Of course you will probably want the output from the model.

In [26]:
# Create a single run 
fresh_run = client.beta.threads.runs.create_and_poll(
    assistant_id=assistant.id,
    thread_id=thread.id,
)

# Retrieve messages from the thread
messages = client.beta.threads.messages.list(thread_id=thread.id)

# Get the latest assistant message 
latest_assistant_message = None
for message in messages.data:
    if message.role == 'assistant' and message.run_id == fresh_run.id:
        latest_assistant_message = message
        break

# Print the latest response
if latest_assistant_message:
    print("Output:\n" + latest_assistant_message.content[0].text.value) 
else:
    print("No assistant message found for the fresh run.")

Output:
Penguins are flightless birds that have a highly adapted sense for aquatic life. Originating mostly in the Southern Hemisphere, especially in Antarctica, these birds are known for their distinctive black and white plumage which acts as camouflage while swimming. The white belly blends with the bright surface water, deceiving predators from below, and the black back masks them against the darker ocean depths when viewed from above.

Penguins are excellent divers, adapting to capture their prey underwater, where they eat a variety of marine organisms including fish, squid, and krill. When on land, they have an upright stance and can often be seen waddling or sliding on their bellies across the ice. Penguins are also sociable animals, forming large colonies that help in mating, nesting, and protecting against predators.

Their breeding behavior is notable: they lay eggs and raise their chicks on land in regions that can be exceedingly harsh. Parents often take turns warming the eg

### Creating a Simple Streaming Run
Instead of having to wait for the entire run to finish before getting a result, it is best to stream output to the user for interactive sessions.

In [ ]:
# Create a new run with the stream option set to True
# Attempt to create a new run
stream_run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    stream=True
)

# Continue processing the new run
for event in stream_run:
    if hasattr(event.data, 'status'):  # Check if 'status' is an attribute of the event data
        print(event.data.id)
        print(event.data.status)
    else:
        print(f"Event ID: {event.data.id} does not have a status attribute.")
        print(event.data.delta)
    print("---------------\n")


### Dealing with Thread Locks

#### Notification of Failed Run
You can't have multiple runs on a thread. To deal with situations where this might occur, you can check to see if a run is already taking place and get feedback on your failed run.

In [ ]:
# create a run to interfere with the other run
interference_run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
        stream=True
    )

# Function to check if there is an active run
def check_active_run(thread_id):
    try:
        # Fetch runs for the thread and check their status
        active_runs = client.beta.threads.runs.list(thread_id=thread_id)
        for run in active_runs.data:
            if run.status in ["in_progress", "queued"]:
                return run.id
        return None
    except OpenAIError as e:
        print(f"Failed to check runs: {str(e)}")
        return None

# Get the active run ID, if any
active_run_id = check_active_run(thread.id)
if active_run_id:
    print(f"Thread already has an active run: {active_run_id}. Please wait until it completes.")
else:
    try:
        # Attempt to create a new run if no active run exists
        stream_run = client.beta.threads.runs.create(
            thread_id=thread.id,
            assistant_id=assistant.id,
            stream=True
        )

        # Continue processing the new run
        for event in stream_run:
            if hasattr(event.data, 'status'):  # Check if 'status' is an attribute of the event data
                print(event.data.id)
                print(event.data.status)
            else:
                print(f"Event ID: {event.data.id} does not have a status attribute.")
                print(event.data.delta)  # Ensure that event.data.delta exists or handle it similarly
            print("---------------\n")

    except OpenAIError as e:  # Handle generic OpenAIError if BadRequestError is not explicitly available
        print(f"Error occurred: {e}")
        if "already has an active run" in str(e):
            print("A run is already active on this thread. Please wait until it completes.")

#### Waiting Your Turn
A better strategy is to check to see if there is already a run going on and just poll until the run is complete then do your run afterward

In [ ]:
import time

# create a run to interfere with the other run
interference_run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
        stream=True
    )

# Function to check if there is an active run
def check_active_run(thread_id):
    try:
        # Fetch runs for the thread and check their status
        active_runs = client.beta.threads.runs.list(thread_id=thread_id)
        for run in active_runs.data:
            if run.status in ["in_progress", "queued"]:
                return True
        return False
    except OpenAIError as e:
        print(f"Failed to check runs: {str(e)}")
        return False

# Wait until there is no active run
while check_active_run(thread.id):
    print("Waiting for the existing run to complete...")
    time.sleep(10)  # Wait for 10 seconds before checking again

# Once no active runs are detected, proceed to create a new run
try:
    print("Creating a new run...")
    stream_run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
        stream=True
    )

    # Continue processing the new run
    for event in stream_run:
        if hasattr(event.data, 'status'):  # Check if 'status' is an attribute of the event data
            print(event.data.id)
            print(event.data.status)
        else:
            print(f"Event ID: {event.data.id} does not have a status attribute.")
            print(event.data.delta)  # Ensure that event.data.delta exists or handle it similarly
        print("---------------\n")

except OpenAIError as e:
    print(f"Error occurred: {e}")


## Modifying Assistants and Threads with Runs
You can make changes in real-time to your Runs that override or modify existing setting on Assistants and Threads.

### Runs and Assistants
Let's see how we can make changes to the Assistant settings for our Runs.

In [25]:
# Show the Assistant instructions
print(assistant.instructions)
print("\n")

# Create a run to add to Assistant
additional_instruction_run = client.beta.threads.runs.create_and_poll(
    assistant_id=assistant.id,
    thread_id=thread.id,
    additional_instructions="That speaks like a pirate.",
)

print(additional_instruction_run.instructions)
print("\n")

# Retrieve messages from the thread
messages = client.beta.threads.messages.list(thread_id=thread.id)

# Get the latest assistant message 
latest_assistant_message = None
for message in messages.data:
    if message.role == 'assistant' and message.run_id == additional_instruction_run.id:
        latest_assistant_message = message
        break

# Print the latest response
if latest_assistant_message:
    print("Output:\n" + latest_assistant_message.content[0].text.value.strip()) 
else:
    print("No assistant message found for the run.")

# add some space
print("\n")


# print the original assistant instructions
print(assistant.instructions)

You are a helpful assistant.


You are a helpful assistant. That speaks like a pirate.


Output:
Ahoy there! Penguins be stout-bodied seabirds that make their homes in the Southern Hemisphere. They be splendiferous swimmers with wings turned flippers that propel 'em through the salty deep like no other. Decked out in their iconic black and white plumage, they cut a fine form, appearing in formal wear to any who glance their way.

These buccaneers of the chilly waters dine heartily on seafood such as krill, fish, and the occasional squid, diving deep and bobbing back up like corks in the ocean vast. On land, they might waddle comedically, but in the brine, they be as graceful as any sea creature can claim to be.

With colonies that rival the size of cities, sprawling on ice or rocky island coasts, penguins aren't just about the individual life; they band together for survival in the harsh climes of the Antarctic or any coastal area liberal with its chill. Be it the towering Emperor peng